In [4]:
import gym
import pandas as pd
import numpy as np
import random

class QLearner(object):
    def __init__(self,
                 num_states=100,
                 num_actions=4,
                 alpha=0.2,
                 gamma=0.9,
                 random_action_rate=0.5,
                 random_action_decay_rate=0.99):
        self.num_states = num_states
        self.num_actions = num_actions
        self.alpha = alpha
        self.gamma = gamma
        self.random_action_rate = random_action_rate
        self.random_action_decay_rate = random_action_decay_rate
        self.state = 0
        self.action = 0
        self.qtable = np.random.uniform(low=-1, high=1, size=(num_states, num_actions))

    def set_initial_state(self, state):
        """
        @summary: Sets the initial state and returns an action
        @param state: The initial state
        @returns: The selected action
        """
        self.state = state
        self.action = self.qtable[state].argsort()[-1]
        return self.action

    def move(self, state_prime, reward):
        """
        @summary: Moves to the given state with given reward and returns action
        @param state_prime: The new state
        @param reward: The reward
        @returns: The selected action
        """
        alpha = self.alpha
        gamma = self.gamma
        state = self.state
        action = self.action
        qtable = self.qtable

        choose_random_action = (1 - self.random_action_rate) <= np.random.uniform(0, 1)

        if choose_random_action:
            action_prime = random.randint(0, self.num_actions - 1)
        else:
            action_prime = self.qtable[state_prime].argsort()[-1]

        self.random_action_rate *= self.random_action_decay_rate

        qtable[state, action] = (1 - alpha) * qtable[state, action] + alpha * (reward + gamma * qtable[state_prime, action_prime])

        self.state = state_prime
        self.action = action_prime

        return self.action


import tempfile

random.seed(0)
env = gym.make('CartPole-v0')
experiment_filename = tempfile.gettempdir() + '/cartpole-experiment-1'
env = gym.wrappers.Monitor(env, experiment_filename, force=True)
goal_average_steps = 195
max_number_of_steps = 200
number_of_iterations_to_average = 100

number_of_features = env.observation_space.shape[0]
last_time_steps = np.ndarray(0)

cart_position_bins = pd.cut([-2.4, 2.4], bins=10, retbins=True)[1][1:-1]
pole_angle_bins = pd.cut([-2, 2], bins=10, retbins=True)[1][1:-1]
cart_velocity_bins = pd.cut([-1, 1], bins=10, retbins=True)[1][1:-1]
angle_rate_bins = pd.cut([-3.5, 3.5], bins=10, retbins=True)[1][1:-1]

def build_state(features):
    return int("".join(map(lambda feature: str(int(feature)), features)))

def to_bin(value, bins):
    return np.digitize(x=[value], bins=bins)[0]

learner = QLearner(num_states=10 ** number_of_features,
                   num_actions=env.action_space.n,
                   alpha=0.2,
                   gamma=1,
                   random_action_rate=0.5,
                   random_action_decay_rate=0.99)

for episode in range(1000000):
    observation = env.reset()
    cart_position, pole_angle, cart_velocity, angle_rate_of_change = observation
    state = build_state([to_bin(cart_position, cart_position_bins),
                         to_bin(pole_angle, pole_angle_bins),
                         to_bin(cart_velocity, cart_velocity_bins),
                         to_bin(angle_rate_of_change, angle_rate_bins)])
    action = learner.set_initial_state(state)

    while True:

        observation, reward, done, info = env.step(action)
        cart_position, pole_angle, cart_velocity, angle_rate_of_change = observation

        state_prime = build_state([to_bin(cart_position, cart_position_bins),
                                   to_bin(pole_angle, pole_angle_bins),
                                   to_bin(cart_velocity, cart_velocity_bins),
                                   to_bin(angle_rate_of_change, angle_rate_bins)])

        action = learner.move(state_prime, reward)

        if done:
            break
        state = state_prime

    if last_time_steps.mean() > goal_average_steps:
        print("Goal reached!")
        print("Episodes before solve: ", episode + 1)
        print(u"Best 100-episode performance {} {} {}".format(last_time_steps.max(),
                                                              unichr(177),  # plus minus sign
                                                              last_time_steps.std()))


[2017-03-24 19:26:10,320] Making new env: CartPole-v0
[2017-03-24 19:26:10,322] Clearing 2 monitor files from previous run (because force=True was provided)
[2017-03-24 19:26:10,325] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.2.12.video000000.mp4


NameError: name 'base' is not defined

In [ ]:
env.videos[-1][0]

In [ ]:
import io
import json
import base64

from IPython.display import HTML

video_path, meta_path = env.videos[-1]

video = io.open(video_path, 'r+b').read()
encoded = base64.b64encode(video)

with open(meta_path) as data_file:    
    meta = json.load(data_file)

html_tag = """
<h2>{0}<h2/>
<video width="960" height="540" controls>
    <source src="data:video/mp4;base64,{1}" type="video/mp4" />
</video>"""
strm = html_tag.format('Episode ' + str(meta['episode_id']), encoded.decode('ascii'))
HTML(data=strm)

In [3]:
strm = ''
for video_path, meta_path in env.videos:
    video = io.open(video_path, 'r+b').read()
    encoded = base64.b64encode(video)
    
    with open(meta_path) as data_file:    
        meta = json.load(data_file)

    html_tag = """
    <h2>{0}<h2/>
    <video width="960" height="540" controls>
        <source src="data:video/mp4;base64,{1}" type="video/mp4" />
    </video>"""
    strm += html_tag.format('Episode ' + str(meta['episode_id']), encoded.decode('ascii'))
HTML(data=strm)

NameError: name 'env' is not defined

In [4]:
import gym

env = gym.make('CartPole-v0')
env = gym.wrappers.Monitor(env, '/tmp/cartpole-experiment-1', force=True)
observation = env.reset()
for t in range(100):
#    env.render()
    print(observation)
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done:
        print("Episode finished after {} timesteps".format(t+1))
        break

env.monitor.close()


[2017-03-24 20:15:31,104] Making new env: CartPole-v0
[2017-03-24 20:15:31,106] Clearing 2 monitor files from previous run (because force=True was provided)
[2017-03-24 20:15:31,107] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.2.12.video000000.mp4


NameError: name 'base' is not defined

In [8]:
from pyglet.gl import *

NameError: name 'base' is not defined